This notebook shows how to use optimum-benchmark to benchmark LLMs. It  focuses on benchmarking quantization algorithms for Mistral 7B.

We need to install the following packages. bitsandbytes, auto-gptq and autoawq are only necessary if you benchmark models quantized with these algorithms.

In [ ]:
!python -m pip install git+https://github.com/huggingface/optimum-benchmark.git
!pip install bitsandbytes
!pip install auto-gptq
!pip install autoawq
!pip install --upgrade transformers #Google Colab doesn't use by default the last version of Transformers

  Cloning https://github.com/huggingface/optimum-benchmark.git to /tmp/pip-req-build-w7nad0pk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum-benchmark.git /tmp/pip-req-build-w7nad0pk
  Resolved https://github.com/huggingface/optimum-benchmark.git to commit ef70214a33902d33896d4edd663e08480682c05f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/RadeonOpenCompute/pyrsmi.git to /tmp/pip-install-ixjgfm_5/pyrsmi_fe0ad5f50f4b4933bd5593e62a4c335e
  Running command git clone --filter=blob:none --quiet https://github.com/RadeonOpenCompute/pyrsmi.git /tmp/pip-install-ixjgfm_5/pyrsmi_fe0ad5f50f4b4933bd5593e62a4c335e
  Resolved https://github.com/RadeonOpenCompute/pyrsmi.git to commit 0c27f5c5bc6bbb6c3fd60b57cab67eb92f44b8e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 3.0 MB/s eta

Define the configuration for optimum-benchmark.

Here we benchmark for inference, using different batch sizes, Mistral 7B loaded as fp16.
If you run this notebook on Google Colab, you will need the A100 only for this part. The following benchmarks would run on the T4.

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.input_shapes.batch_size: 1,2,4,8,16 #we will try all these batch sizes

experiment_name: fp16-batch_size(${benchmark.input_shapes.batch_size})-sequence_length(${benchmark.input_shapes.sequence_length})-new_tokens(${benchmark.new_tokens})
model: mistralai/Mistral-7B-v0.1 #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
"""

with open("mistral_7b_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_ob --multirun

2024-01-16 04:32:34.977174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 04:32:34.977223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 04:32:34.978673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 04:32:36.174801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-16 04:32:39,634][HYDRA] Launching 5 jobs locally
[2024-01-16 04:32:39,635][HYDRA] 	#0 : benchmark.input_shapes.batch_size=1
[2024-01-16 04:32:39,799][launcher][INFO] - Configuring process launcher
[2024-01-16 04:32:39,799][process][

Benchmarking BNB's NF4 quantization without double quantization

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.input_shapes.batch_size: 1,2,4,8,16 #we will try all these batch sizes

experiment_name: bnb-batch_size(${benchmark.input_shapes.batch_size})-sequence_length(${benchmark.input_shapes.sequence_length})-new_tokens(${benchmark.new_tokens})
model: mistralai/Mistral-7B-v0.1 #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16
  quantization_scheme: bnb
  quantization_config:
    load_in_4bit: true
    bnb_4bit_compute_dtype: float16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
"""

with open("mistral_7b_bnb_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_bnb_ob --multirun

2024-01-16 04:46:52.647047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 04:46:52.647091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 04:46:52.648589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 04:46:53.830690: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-16 04:46:57,283][HYDRA] Launching 5 jobs locally
[2024-01-16 04:46:57,283][HYDRA] 	#0 : benchmark.input_shapes.batch_size=1
[2024-01-16 04:46:57,450][launcher][INFO] - Configuring process launcher
[2024-01-16 04:46:57,451][process][

Benchmarking BNB's NF4 with double quantization

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.input_shapes.batch_size: 1,2,4,8,16 #we will try all these batch sizes

experiment_name: bnb_dq-batch_size(${benchmark.input_shapes.batch_size})-sequence_length(${benchmark.input_shapes.sequence_length})-new_tokens(${benchmark.new_tokens})
model: mistralai/Mistral-7B-v0.1 #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16
  quantization_scheme: bnb
  quantization_config:
    load_in_4bit: true
    bnb_4bit_compute_dtype: float16
    bnb_4bit_use_double_quant: true

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
"""

with open("mistral_7b_bnb_dq_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_bnb_dq_ob --multirun

2024-01-16 05:08:43.112464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 05:08:43.112511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 05:08:43.113993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 05:08:44.311961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-16 05:08:47,787][HYDRA] Launching 5 jobs locally
[2024-01-16 05:08:47,787][HYDRA] 	#0 : benchmark.input_shapes.batch_size=1
[2024-01-16 05:08:47,955][launcher][INFO] - Configuring process launcher
[2024-01-16 05:08:47,956][process][

Benchmarking AWQ

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.input_shapes.batch_size: 1,2,4,8,16 #we will try all these batch sizes

experiment_name: awq-batch_size(${benchmark.input_shapes.batch_size})-sequence_length(${benchmark.input_shapes.sequence_length})-new_tokens(${benchmark.new_tokens})
model: kaitchup/Mistral-7B-awq-4bit #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
"""

with open("mistral_7b_awq_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_awq_ob --multirun

2024-01-16 04:17:09.135766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 04:17:09.135815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 04:17:09.137536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 04:17:10.296696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-16 04:17:13,735][HYDRA] Launching 5 jobs locally
[2024-01-16 04:17:13,735][HYDRA] 	#0 : benchmark.input_shapes.batch_size=1
[2024-01-16 04:17:13,900][launcher][INFO] - Configuring process launcher
[2024-01-16 04:17:13,900][process][

Benchmarking GPTQ

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.input_shapes.batch_size: 1,2,4,8,16 #we will try all these batch sizes

experiment_name: gptq-batch_size(${benchmark.input_shapes.batch_size})-sequence_length(${benchmark.input_shapes.sequence_length})-new_tokens(${benchmark.new_tokens})
model: flozi00/Mistral-7B-v0.1-4bit-autogptq #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
"""

with open("mistral_7b_gptq_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_gptq_ob --multirun

2024-01-16 05:34:18.710547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 05:34:18.710597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 05:34:18.712068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 05:34:19.882113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-16 05:34:23,360][HYDRA] Launching 5 jobs locally
[2024-01-16 05:34:23,360][HYDRA] 	#0 : benchmark.input_shapes.batch_size=1
[2024-01-16 05:34:23,525][launcher][INFO] - Configuring process launcher
[2024-01-16 05:34:23,526][process][

# Benchmarking QLoRA training

In [ ]:
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: training # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments_training/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments_training/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID
  sweeper:
    params:
      benchmark.training_arguments.per_device_train_batch_size: 1,2,4,8 #we will try all these batch sizes

experiment_name: qlora-batch_size(${benchmark.training_arguments.per_device_train_batch_size})
model: mistralai/Mistral-7B-v0.1 #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  no_weights: true
  torch_dtype: float16 #The model will be loaded with fp16
  #peft_model: kaitchup/Mistral-7B-v0.1-SFT-ultrachat
  peft_strategy: lora
  peft_config:
    task_type: CAUSAL_LM
  quantization_scheme: bnb
  quantization_config:
    load_in_4bit: true
    bnb_4bit_compute_dtype: float16
    bnb_4bit_use_double_quant: true



benchmark:
  memory: true
  warmup_steps: 40
  dataset_shapes:
    dataset_size: 160
    sequence_length: 256
  training_arguments:
    max_steps: 140
    per_device_train_batch_size: 1
"""

with open("mistral_7b_qlora_ob.yaml", 'w') as f:
  f.write(YAML_DEFAULT)

In [ ]:
!optimum-benchmark --config-dir ./ --config-name mistral_7b_qlora_ob --multirun

2024-01-15 14:27:41.423277: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 14:27:41.423331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 14:27:41.424639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 14:27:42.672549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-01-15 14:27:46,109][HYDRA] Launching 4 jobs locally
[2024-01-15 14:27:46,109][HYDRA] 	#0 : benchmark.training_arguments.per_device_train_batch_size=1
[2024-01-15 14:27:46,283][launcher][INFO] - Configuring process launcher
[2024-01-15 

# Generating plots

We will use the script prepared by optimum-benchmark:

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/optimum-benchmark/ef70214a33902d33896d4edd663e08480682c05f/examples/running-mistrals/report.py

--2024-01-16 05:49:42--  https://raw.githubusercontent.com/huggingface/optimum-benchmark/ef70214a33902d33896d4edd663e08480682c05f/examples/running-mistrals/report.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8097 (7.9K) [text/plain]
Saving to: ‘report.py’

report.py           100%[===================>]   7.91K  --.-KB/s    in 0s      

2024-01-16 05:49:42 (68.7 MB/s) - ‘report.py’ saved [8097/8097]



In [ ]:
!pip install flatten-dict

In [ ]:
!python report.py -e experiments

/content/report.py:38: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  inference_report = pd.concat(inference_reports, axis=0, ignore_index=True)
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃           ┃           ┃            ┃           ┃    Forward ┃           ┃   Generate ┃           ┃
┃           ┃           ┃    Forward ┃   Forward ┃       Peak ┃  Generate ┃       Peak ┃           ┃
┃ Experime… ┃     Batch ┃    Latency ┃ Throughp… ┃     Memory ┃ Throughp… ┃     Memory ┃ Quantiza… ┃
┃ Name      ┃      Size ┃        (s) ┃ (samples… ┃       (MB) ┃ (tokens/… ┃       (MB) ┃    Scheme ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ fp16-bat… │        16 │   7.21e-01 │     22.20 │      19947 │    311.00 │      24221 │      fp16 │
├───────────┼───────